# Machine Learning Experiment \#0

First time messing around with neural networks. Attempting to use time series data as input, having the model predict the next value based on the previous N values. Fake data is constructed so that the probability distribution for the next value depends on the previous N values (or a subset of them) in an obvious way.

## Findings

Difficult to disentangle input variables that are highly correlated. Two examples of this:
* Raw random walk data have strong correlation between adjacent values (since adjacent values are always within +/- 1). Use diff instead of the raw values to obtain independent inputs.
* Model data used a probability depending entirely on the previous tick, i.e. if the previous tick was an up-tick, the current tick has an 80% probability of being an up-tick as well. Ideally the ML would identify this and give a weight of zero to all but the most recent tick. However, the second-last tick is extremely likely to be in the same direction as the last, so it is difficult for the model to tell the difference.
Because of this, I was not able to train a model (not quickly anyway) to understand the exact rule and guess the exact probability, but its guess would still almost always lean towards the more likely outcome (up or down).

The reward used for training was based on the outcome +1 or -1 rather than the probability, so whenever the less likely outcome occured, the model would receive negative reinforcement. This amount of noise leads to slower convergence.

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time

In [ ]:
class ModelMarket:
    def __init__(self):
        self.pre_data = [ii for ii in range(1000)]
        self.price = 100000 * [None]
        self.p = 100000 * [None]
        self.d = 100000 * [None]
        self.curr_index = 0
        
    def GetNPrevious(self, N, index=None):
        if not index:
            index = self.curr_index
            
        if index >= N:
            return self.price[index-N:index]
        else:
            excess = N-index
            return self.pre_data[-excess:] + self.price[0:index]
        
    def GetLastP(self):
        return self.p[self.curr_index-1]
    def GetLastD(self):
        return self.d[self.curr_index-1]
        
    def Tick(self):
        last2 = self.GetNPrevious(2)
        
        self.p[self.curr_index] = 0.5 + 0.1*np.diff(last2)[0]
        if np.random.rand() < self.p[self.curr_index]:
            self.price[self.curr_index] = last2[1]+1
            self.d[self.curr_index] = 1
        else:
            self.price[self.curr_index] = last2[1]-1
            self.d[self.curr_index] = -1
            
        self.curr_index += 1

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras import optimizers
from keras import regularizers
import random
import numpy as np
import pandas as pd
from operator import add


class DQNAgent(object):

    def __init__(self):
        self.model = self.network()

    def get_state(self, market):
        
        state = market.GetNPrevious(11)
        state = np.asarray(state)
        state = np.diff(state)
        
        return state

    def network(self, weights=None):
        model = Sequential()
        
#         model.add(Dense(units=5, activation='relu', input_dim=10,
#                         kernel_regularizer=regularizers.l1(0.01),
#                         activity_regularizer=regularizers.l1(0.01)))
        model.add(Dense(units=1, activation='linear', input_dim=10,
                kernel_regularizer=regularizers.l1(0.01),
                activity_regularizer=regularizers.l1(0.00)))

#        model.add(Dense(units=1, activation='linear'))
#        model.add(Dense(units=1, activation='linear', input_dim=3))
    
        opt = optimizers.SGD()
        model.compile(loss='mse', optimizer=opt)

        if weights:
            model.load_weights(weights)
        return model

    def train(self, state, target):
        hist = self.model.fit(state, target, epochs=1, verbose=0, batch_size=30)
        return hist.history['loss'][0]

In [ ]:
def run(chart = None):
    agent = DQNAgent()
    market = ModelMarket()
    
    iterations = 5000
    counter = 0
    
    states = iterations * [None]
    targets = iterations * [None]
    
    x = [ii for ii in range(iterations)]
    y = iterations * [None]
    
    while counter < iterations:
        
        state = agent.get_state(market)
        market.Tick()
        target = np.asarray([market.GetLastD()])
        
        guess = agent.model.predict(state.reshape(1,10))
                
        states[counter] = state
        targets[counter] = target
        if counter > 0 and counter % 50 == 0:
            agent.model.optimizer.learning_rate = 0.3
            indices = np.asarray(random.sample([ii for ii in range(counter)], 29))

            state_in = np.asarray([state] + [states[ii] for ii in indices])
            target_in = np.asarray([target] + [targets[ii] for ii in indices])
            
            y[counter] = agent.train(state_in, target_in)
            
#             print("weights:")
#             for layer in agent.model.layers:
#                 print(layer.get_weights())

        print(str(market.GetLastD()) + "  " + str(guess))
        
        counter += 1
        
    if chart:
        chart.chart.data[0].x = x
        chart.chart.data[0].y = y
        
run(chart)

In [ ]:
class Chart:
    def __init__(self):
        data = [dict(
                type = 'scatter',
                x = [],
                y = [],
                mode='markers')]
        layout = dict(
                title = 'ML Convergence',
                xaxis = dict(
                    title = dict(
                        text = 'Time'
                    )
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Loss'
                    ),
                    range = ([0,10])
                ))
        
        self.chart = go.FigureWidget( data=data, layout=layout)
        
chart = Chart()
chart.chart